In [1]:
import uuid
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from alchemy import Referee, Base, Proposal
import lorem
import json
from shutil import copyfile
import os

import _pickle as pickle

/home/wkerzend/miniconda3/envs/deepthought/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# getting the refereeing done:
base_path = '/home/wkerzend/dpr_data/'
assert os.path.exists(base_path)
opc_refereeing_matrices = pickle.load(open(os.path.join(base_path, 'ref_matrix_abcd_worked.pkl'), 'rb'))
deepthought_refereeing_matrix = pd.read_hdf(os.path.join(base_path, 'deepthought_refereeing_no_tele.h5'))
full_dpr_list = pd.read_hdf(os.path.join(base_path, 'dpr_list.h5'), 'dpr_list')
refereeing_list = pd.DataFrame(index=full_dpr_list.index, columns=np.arange(8), data='NA')
for cat, ref_matrix in opc_refereeing_matrices.items():
    print('ingesting cat', cat)
    for author_id, ref_row in ref_matrix.iterrows():
        refereeing_list.loc[author_id] = ref_matrix.columns[ref_row.values.astype(bool)]
print('ingesting deepthought refereeing matrix')
for author_id, ref_row in deepthought_refereeing_matrix.iterrows():
    refereeing_list.loc[author_id] = deepthought_refereeing_matrix.columns[ref_row.values.astype(bool)]

/home/wkerzend/miniconda3/envs/deepthought/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


ingesting cat A
ingesting cat B
ingesting cat C
ingesting cat D
ingesting deepthought refereeing matrix


In [3]:
with open('credentials.json') as f:
    credentials = json.load(f)
runEnv = credentials['env'] # development = sqlite; production = mysql
dsn = credentials[runEnv]['dbo'] # in the format <engine>://<connection_string>

In [4]:
if runEnv == 'test':
    !rm dt_opc_test.db
else:
    engine = create_engine(dsn)
    connection = engine.connect()
    connection.execute('drop table if exists reviews')
    connection.execute('drop table if exists proposals')
    connection.execute('drop table if exists referees')
    connection.close()
!rm proposals/*.pdf

rm: cannot remove ‘proposals/*.pdf’: No such file or directory


In [5]:
## generate 50 referees
#ref_num = 50
P103_path = '/home/wkerzend/dpr_data/P103/'
referee_ids = full_dpr_list.dpr_id#[str(uuid.uuid4()).split('-')[0] for i in range(50)]

## generate 50 proposals

#prop_num = 50
#prop_idx = np.arange(1000)
#np.random.shuffle(prop_idx)
#proposal_ids = ['103.x-{0:04d}'.format(rand_id) for rand_id in prop_idx[:50]]
proposal_ids = full_dpr_list.PCODE.tolist()
proposal_titles = full_dpr_list.TITLE.tolist() #[lorem.sentence() for i in range(50)]
proposal_abstracts = [open(os.path.join(P103_path, 'LATEX', pcode +'_abstract.txt'), 'r', encoding='utf8').read() for pcode in proposal_ids]
proposal_abstracts = [item.encode('latin-1', errors='replace') for item in proposal_abstracts]
#proposal_abstracts
#proposal_abstract = [open(fname) for fname in proposal_ids]

In [6]:
dpr_id2author_id = {value:key for key, value in referee_ids.to_dict().items()}

In [7]:
# an Engine, which the Session will use for connection
# resources
#engine = create_engine('sqlite:///dt_opc_test.db')
engine = create_engine(dsn)

# create a configured "Session" class
Session = sessionmaker(bind=engine)

# create a Session
session = Session()

In [8]:
Base.metadata.create_all(engine)

In [9]:
for ref_id in referee_ids:
    session.add(Referee(uuid=ref_id))

In [10]:
for i, prop_id in enumerate(proposal_ids):
    session.add(Proposal(eso_id=prop_id, title=proposal_titles[i], 
                         abstract=proposal_abstracts[i]))
    # make a dummy pdf file:
    #copyfile('proposals/placeholder.pdf.master', 'proposals/'+prop_id+'.pdf')    

In [11]:
session.commit()

/home/wkerzend/miniconda3/envs/deepthought/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'eso_id' at row 1")
  result = self._query(query)


In [ ]:
prop_id

In [30]:
for ref in session.query(Referee):
    cur_author_id = dpr_id2author_id[ref.uuid]
    for prop_id in refereeing_list.loc[cur_author_id]:
        prop = session.query(Proposal).filter_by(eso_id=prop_id).one()
    #for prop in np.random.choice(session.query(Proposal).all(), size=8).tolist():
        ref.proposals.append(prop)

NoResultFound: No row was found for one()

In [11]:
session.commit()

In [12]:
np.random.seed = 25081980
for ref in session.query(Referee):
    # 20% of referees are finished
    if np.random.random() > 0.8: 
        ref.accepted_tou = True
        for review in ref.reviews:
            review.comment = lorem.paragraph()
            review.score = np.random.uniform(1, 5)
            review.ref_knowledge = np.random.randint(3) + 1
            if np.random.random() > 0.8:
                review.conflicted = np.random.randint(2) + 1
        ref.finalized_submission = True
    # 60% have accepted tou and done some work; remaining 20% haven't accepted tou or done work yet
    elif np.random.random() > 0.2:
        ref.accepted_tou = True
        for review in ref.reviews:
            if np.random.random() > 0.5:
                review.comment = lorem.paragraph()
            if np.random.random() > 0.5:
                review.score = np.random.uniform(-1, 7)
            if np.random.random() > 0.5:
                review.ref_knowledge = np.random.randint(3) + 1
            if np.random.random() > 0.8:
                review.conflicted = np.random.randint(2) + 1


In [13]:
session.commit()

In [14]:
if runEnv == 'test':
    !cp dt_opc_test.db dt_opc_test.db.bak

In [15]:
test_ref = session.query(Referee).first()